# Pre-Requisites

In [ ]:
!pip3 install beautifulsoup4 pandas

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import requests

/home/rezamohideen/myProjects/semantic-search/ENV/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nocd = "https://www.treatmyocd.com"
all_articles = "https://www.treatmyocd.com/learn/blog"
blog_categories = set()

resp = requests.get(all_articles)
soup = bs(resp.text, "html.parser")
for tag in soup.find_all(): 
    if tag.name == 'a' and 'href' in tag.attrs:
        if 'blog/category' in tag['href']:
            blog_categories.add(tag['href'])

articles = set()
for cat in tqdm(blog_categories):
    resp = requests.get(nocd + cat)
    soup = bs(resp.text, "html.parser")
    for tag in soup.find_all(): 
        if tag.name == 'a' and 'href' in tag.attrs:
            if 'blog/' in tag['href']:
                articles.add(tag['href'])
                
print(len(articles))

100%|██████████| 3/3 [00:00<00:00,  3.84it/s]

241


In [187]:
blog_text = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', None]
paragraph_headings = ['h2', 'h3', 'h4', 'h5', 'h6']

def extract_text_v2(soup, article_name):
    article = soup.select('article')
    article = article[0].select('div[class*="Post"]')
    title = [((article[0].find_all('h1'))[0].text, 'h1', 'title', article_name)]
    body = soup.select('div[class*="Body"]')

    curr_paragraph = None
    lines = []
    for content in body[0].contents:
        if content.name in blog_text:
            try:
                # get text contents
                line = content.getText()
                # ignore \n characters and empty spaces
                if len(line.split()) > 1:
                    # append styles like bold (<em>), links (<a>), etc. to previous ine
                    if content.name not in blog_text:
                        lines[-1] += line
                        continue
                    # grab paragraph name
                    if content.name in paragraph_headings:
                        curr_paragraph = line

                    lines.append((line, content.name, curr_paragraph, article_name))
                    assert len(lines[-1]) == 4, f"Line should only contain 4 objects; {article_name}"
            except Exception:
                lines.append(content)

            

    # body = (body[0].text).split("\n")

    return title + lines

In [188]:

# for article in articles:
#     resp = requests.get(nocd + article)
#     soup = bs(resp.text, "html.parser")
#     for tag in soup.find_all():
#         if tag.name == 'article':
#             for t in tag.find_all('div'):
#                 print(t.attrs)

blogs = []
for article in tqdm(articles):
    if article not in blog_categories:
        # print(article)
        resp = requests.get(nocd + article)
        soup = bs(resp.text, "html.parser")
        blog = extract_text_v2(soup, article)
        blogs += blog



100%|██████████| 241/241 [02:50<00:00,  1.41it/s]


In [191]:
import pandas as pd 

df = pd.DataFrame(blogs, columns=['text', 'tag', 'paragraph', 'article'])

5233